In [ ]:
import pandas as pd
import nltk
import spacy
from gensim import corpora, models
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
import re
from numpy import argmax

In [ ]:
# Загрузка данных
df = pd.read_csv("/content/cleaned_museums.csv")

# Предварительная обработка текста
nltk.download('punkt_tab')
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('russian')

def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    tokens = nltk.word_tokenize(text)
    tokens = [w for w in tokens if not w in stop_words and len(w) > 2]
    return tokens

df['processed_text'] = df['Description'].apply(preprocess_text)


# ЧАСТЬ А: Тематическое моделирование

# 1. LDA
vectorizer = CountVectorizer(max_df=0.95, min_df=2)
doc_term_matrix = vectorizer.fit_transform(df['Description'])
lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(doc_term_matrix)

# Вывод топ-слов для каждой темы LDA
feature_names = vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(lda.components_):
    print(f"Тема {topic_idx + 1}:")
    top_words = [feature_names[i] for i in topic.argsort()[:-10 - 1:-1]]
    print(top_words)
    print()


# 2. LSA
vectorizer_lsa = TfidfVectorizer(max_df=0.95, min_df=2)
doc_term_matrix_lsa = vectorizer_lsa.fit_transform(df['Description'])
lsa = TruncatedSVD(n_components=5, random_state=42)
lsa.fit(doc_term_matrix_lsa)

# Вывод топ-слов для каждой темы LSA (аналогично LDA)
feature_names_lsa = vectorizer_lsa.get_feature_names_out()
for topic_idx, topic in enumerate(lsa.components_):
    print(f"Тема LSA {topic_idx + 1}:")
    top_words = [feature_names_lsa[i] for i in topic.argsort()[:-10 - 1:-1]]
    print(top_words)
    print()

# 3. NMF
nmf = NMF(n_components=5, random_state=42) # количество тем - 5
nmf.fit(doc_term_matrix)

# ЧАСТЬ Б: Выделение именованных сущностей

# Загрузка модели spaCy
nlp = spacy.load("en_core_web_sm")

try:
    nlp = spacy.load("ru_core_news_sm")
except OSError:
    print("Модель spaCy 'ru_core_news_sm' не найдена.  Загрузите ее: python -m spacy download ru_core_news_sm")
    exit()


def extract_entities(text):
    doc = nlp(text)
    entities = {}
    for ent in doc.ents:
        if ent.label_ in entities:
            entities[ent.label_].append(ent.text)
        else:
            entities[ent.label_] = [ent.text]
    return entities

df['entities'] = df['Description'].apply(extract_entities)


# Анализ именованных сущностей в каждой теме (исправленный код для LDA)
topic_probabilities = lda.transform(doc_term_matrix)
for i in range(lda.n_components):
    print(f"\nТема LDA {i+1}: { [feature_names[j] for j in lda.components_[i].argsort()[:-10 - 1:-1]]}")
    topic_indices = argmax(topic_probabilities, axis=1) == i
    entities_in_topic = []
    for index in df[topic_indices].index:
        entities_in_topic.extend(df['entities'][index])
    print("Сущности в теме:", entities_in_topic)


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Тема 1:
['россии', 'где', 'исторический', 'музей', 'которого', 'самых', 'из', 'европе', 'музеев', 'один']

Тема 2:
['из', 'музей', 'здесь', 'музеев', 'один', 'мира', 'им', 'одно', 'москве', 'коллекций']

Тема 3:
['искусства', 'это', 'центр', 'самых', 'для', 'как', 'из', 'россии', 'музей', 'современного']

Тема 4:
['она', 'не', 'музей', 'но', 'музеем', 'самым', 'является', 'москве', 'котором', 'достопримечательностей']

Тема 5:
['на', 'музей', 'расположенный', 'был', 'москвы', 'от', 'км', 'крупным', 'является', 'самым']

Тема LSA 1:
['музей', 'из', 'самых', 'как', 'не', 'на', 'россии', 'был', 'для', 'музеев']

Тема LSA 2:
['для', 'был', 'музей', 'как', 'на', 'россии', 'по', 'хранения', 'центре', 'искусств']

Тема LSA 3:
['на', 'где', 'расположенный', 'москве', 'одно', 'является', 'самым', 'петра', 'крупным', 'исторический']

Тема LSA 4:
['центр', 'искусства', 'современного', 'году', 'самый', 'москвы', 'россии', 'на', 'более', 'открылся']

Тема LSA 5:
['где', 'между', 'уникальные', 'как'